In [1]:
# data loading
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# for better display of plots
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("../data/final_cohort_main_df.csv")

In [2]:
df.head()

,SUBJECT_ID,HADM_ID,GENDER,age,is_nonsurvivor,ADMITTIME,DISCHTIME,length_of_stay_days,vasopressor,mechanical_ventilation,rrt_dialysis,Lab_Alanine Aminotransferase (ALT)_max_val,Lab_Albumin_max_val,Lab_Alkaline Phosphatase_max_val,Lab_Anion Gap_max_val,Lab_Asparate Aminotransferase (AST)_max_val,Lab_Base Excess_max_val,Lab_Basophils_max_val,Lab_Bicarbonate_max_val,"Lab_Bilirubin, Total_max_val","Lab_Calcium, Total_max_val",Lab_Calculated Total CO2_max_val,Lab_Chloride_max_val,Lab_Creatine Kinase (CK)_max_val,"Lab_Creatine Kinase, MB Isoenzyme_max_val",Lab_Creatinine_max_val,Lab_Eosinophils_max_val,Lab_Free Calcium_max_val,Lab_Glucose_max_val,Lab_Hematocrit_max_val,Lab_Hemoglobin_max_val,Lab_INR(PT)_max_val,Lab_Lactate_max_val,Lab_Lactate Dehydrogenase (LD)_max_val,Lab_Lymphocytes_max_val,Lab_MCH_max_val,Lab_MCHC_max_val,Lab_MCV_max_val,Lab_Magnesium_max_val,Lab_Monocytes_max_val,Lab_Neutrophils_max_val,Lab_Oxygen Saturation_max_val,Lab_PT_max_val,Lab_PTT_max_val,Lab_Phosphate_max_val,Lab_Platelet Count_max_val,Lab_Potassium_max_val,Lab_Protein_max_val,Lab_RDW_max_val,Lab_Red Blood Cells_max_val,Lab_Sodium_max_val,Lab_Specific Gravity_max_val,Lab_Temperature_max_val,Lab_Troponin T_max_val,Lab_Urea Nitrogen_max_val,Lab_White Blood Cells_max_val,Lab_pCO2_max_val,Lab_pH_max_val,Lab_pO2_max_val,Lab_Alanine Aminotransferase (ALT)_mean_val,Lab_Albumin_mean_val,Lab_Alkaline Phosphatase_mean_val,Lab_Anion Gap_mean_val,Lab_Asparate Aminotransferase (AST)_mean_val,Lab_Base Excess_mean_val,Lab_Basophils_mean_val,Lab_Bicarbonate_mean_val,"Lab_Bilirubin, Total_mean_val","Lab_Calcium, Total_mean_val",Lab_Calculated Total CO2_mean_val,Lab_Chloride_mean_val,Lab_Creatine Kinase (CK)_mean_val,"Lab_Creatine Kinase, MB Isoenzyme_mean_val",Lab_Creatinine_mean_val,Lab_Eosinophils_mean_val,Lab_Free Calcium_mean_val,Lab_Glucose_mean_val,Lab_Hematocrit_mean_val,Lab_Hemoglobin_mean_val,Lab_INR(PT)_mean_val,Lab_Lactate_mean_val,Lab_Lactate Dehydrogenase (LD)_mean_val,Lab_Lymphocytes_mean_val,Lab_MCH_mean_val,Lab_MCHC_mean_val,Lab_MCV_mean_val,Lab_Magnesium_mean_val,Lab_Monocytes_mean_val,Lab_Neutrophils_mean_val,Lab_Oxygen Saturation_mean_val,Lab_PT_mean_val,Lab_PTT_mean_val,Lab_Phosphate_mean_val,Lab_Platelet Count_mean_val,Lab_Potassium_mean_val,Lab_Protein_mean_val,Lab_RDW_mean_val,Lab_Red Blood Cells_mean_val,Lab_Sodium_mean_val,Lab_Specific Gravity_mean_val,Lab_Temperature_mean_val,Lab_Troponin T_mean_val,Lab_Urea Nitrogen_mean_val,Lab_White Blood Cells_mean_val,Lab_pCO2_mean_val,Lab_pH_mean_val,Lab_pO2_mean_val,Lab_Alanine Aminotransferase (ALT)_min_val,Lab_Albumin_min_val,Lab_Alkaline Phosphatase_min_val,Lab_Anion Gap_min_val,Lab_Asparate Aminotransferase (AST)_min_val,Lab_Base Excess_min_val,Lab_Basophils_min_val,Lab_Bicarbonate_min_val,"Lab_Bilirubin, Total_min_val","Lab_Calcium, Total_min_val",Lab_Calculated Total CO2_min_val,Lab_Chloride_min_val,Lab_Creatine Kinase (CK)_min_val,"Lab_Creatine Kinase, MB Isoenzyme_min_val",Lab_Creatinine_min_val,Lab_Eosinophils_min_val,Lab_Free Calcium_min_val,Lab_Glucose_min_val,Lab_Hematocrit_min_val,Lab_Hemoglobin_min_val,Lab_INR(PT)_min_val,Lab_Lactate_min_val,Lab_Lactate Dehydrogenase (LD)_min_val,Lab_Lymphocytes_min_val,Lab_MCH_min_val,Lab_MCHC_min_val,Lab_MCV_min_val,Lab_Magnesium_min_val,Lab_Monocytes_min_val,Lab_Neutrophils_min_val,Lab_Oxygen Saturation_min_val,Lab_PT_min_val,Lab_PTT_min_val,Lab_Phosphate_min_val,Lab_Platelet Count_min_val,Lab_Potassium_min_val,Lab_Protein_min_val,Lab_RDW_min_val,Lab_Red Blood Cells_min_val,Lab_Sodium_min_val,Lab_Specific Gravity_min_val,Lab_Temperature_min_val,Lab_Troponin T_min_val,Lab_Urea Nitrogen_min_val,Lab_White Blood Cells_min_val,Lab_pCO2_min_val,Lab_pH_min_val,Lab_pO2_min_val,Vital_ART BP mean_max_val,Vital_Ambulatory aid_max_val,Vital_Arterial BP #2 [Diastolic]_max_val,Vital_Arterial BP #2 [Systolic]_max_val,Vital_Arterial BP Mean_max_val,Vital_Arterial BP Mean #2_max_val,Vital_Arterial BP [Diastolic]_max_val,Vital_Arterial BP [Systolic]_max_val,Vital_Arterial

In [3]:
# to reduce the name of the columns -> we remove the sufix _val
df = df.rename(
    columns=lambda c: c[:-4] if c.endswith('_val') else c
)

vasopressor, mechanical_ventilation and rrt_dialysis contain no missing values because they were previously coalesced to 0.

## Imputation of nans

In [4]:
missing_df = pd.DataFrame({
    'variable': df.columns,
    'n_missing': [df[col].isna().sum() for col in df.columns],
    'pct_missing': [df[col].isna().mean()*100 for col in df.columns]
})

# order by porcentage
missing_df = missing_df.sort_values(by='pct_missing', ascending=False)

print(missing_df.to_string())
missing_df.to_csv("pct_missing.csv", index = False)

                                             variable  n_missing  pct_missing
185                    Vital_Manual BP Mean(calc)_max       3868    99.793602
246                  Vital_Manual BP [Diastolic]_mean       3868    99.793602
186                   Vital_Manual BP [Diastolic]_max       3868    99.793602
305                    Vital_Manual BP Mean(calc)_min       3868    99.793602
306                   Vital_Manual BP [Diastolic]_min       3868    99.793602
245                   Vital_Manual BP Mean(calc)_mean       3868    99.793602
247                   Vital_Manual BP [Systolic]_mean       3867    99.767802
307                    Vital_Manual BP [Systolic]_min       3867    99.767802
187                    Vital_Manual BP [Systolic]_max       3867    99.767802
278               Vital_Arterial BP #2 [Systolic]_min       3865    99.716202
218              Vital_Arterial BP #2 [Systolic]_mean       3865    99.716202
160                     Vital_Arterial BP Mean #2_max       3865

In [5]:
missing_threshold = 85

missing_pct = df.isna().mean() * 100
cols_to_drop = missing_pct[missing_pct >= missing_threshold].index
print(f"{len(cols_to_drop)} columns dropped due to high missing values (>{missing_threshold}%):\n")
print(list(cols_to_drop))

27 columns dropped due to high missing values (>85%):

['Vital_ART BP mean_max', 'Vital_Arterial BP #2 [Diastolic]_max', 'Vital_Arterial BP #2 [Systolic]_max', 'Vital_Arterial BP Mean #2_max', 'Vital_Manual BP Mean(calc)_max', 'Vital_Manual BP [Diastolic]_max', 'Vital_Manual BP [Systolic]_max', 'Vital_Temperature C_max', 'Vital_Temperature Celsius_max', 'Vital_ART BP mean_mean', 'Vital_Arterial BP #2 [Diastolic]_mean', 'Vital_Arterial BP #2 [Systolic]_mean', 'Vital_Arterial BP Mean #2_mean', 'Vital_Manual BP Mean(calc)_mean', 'Vital_Manual BP [Diastolic]_mean', 'Vital_Manual BP [Systolic]_mean', 'Vital_Temperature C_mean', 'Vital_Temperature Celsius_mean', 'Vital_ART BP mean_min', 'Vital_Arterial BP #2 [Diastolic]_min', 'Vital_Arterial BP #2 [Systolic]_min', 'Vital_Arterial BP Mean #2_min', 'Vital_Manual BP Mean(calc)_min', 'Vital_Manual BP [Diastolic]_min', 'Vital_Manual BP [Systolic]_min', 'Vital_Temperature C_min', 'Vital_Temperature Celsius_min']


### Unification of similar or equal variables

Some labels from chartevents or labevents or vital signs were codified with different names resulting in different columns for each label name.

In [ ]:
# fahrenheit -> celsius
def convert_f_to_c(f_val):
    return (f_val - 32) * 5.0/9.0

cols_c = [c for c in df.columns if 'Temperature' in c and ('C_' in c or 'Celsius' in c) and 'max' in c]
cols_f = [c for c in df.columns if 'Temperature' in c and ('F_' in c or 'Fahrenheit' in c) and 'max' in c]

df.drop(columns=cols_c + cols_f, inplace=True)

# we do the same for the minimums and maximums
for suf in ['mean', 'min']:
    df['Vital_Temp_Unified_' + suf] = np.nan
    cols_c = [c for c in df.columns if 'Temperature' in c and ('C_' in c or 'Celsius' in c) and suf in c]
    cols_f = [c for c in df.columns if 'Temperature' in c and ('F_' in c or 'Fahrenheit' in c) and suf in c]
    for c in cols_c:
        df['Vital_Temp_Unified_' + suf] = df['Vital_Temp_Unified_' + suf].fillna(df[c])
    for c in cols_f:
        df['Vital_Temp_Unified_' + suf] = df['Vital_Temp_Unified_' + suf].fillna(df[c].apply(convert_f_to_c))
    
    df.drop(columns=cols_c + cols_f, inplace=True)

In [ ]:
print(df[['Vital_Respiratory Rate (Total)_mean', 'Vital_Resp Rate (Total)_mean', 'Vital_Respiratory Rate_mean']].head(10))

for suf in ['_mean', '_min', '_max']:
    df['Vital_Respiratory Rate (Total) Unified' + suf] = df['Vital_Respiratory Rate (Total)' + suf].combine_first(
        df['Vital_Resp Rate (Total)' + suf]
    )  
    df.drop(columns=['Vital_Resp Rate (Total)'+ suf, 'Vital_Respiratory Rate (Total)' + suf], inplace=True)
    


   Vital_Respiratory Rate (Total)_mean_val  Vital_Resp Rate (Total)_mean_val  \
0                                      NaN                               NaN   
1                                    32.75                               NaN   
2                                    28.50                              28.5   
3                                      NaN                               NaN   
4                                    18.67                               NaN   
5                                      NaN                               NaN   
6                                      NaN                               NaN   
7                                      NaN                               NaN   
8                                    14.80                              14.8   
9                                      NaN                               NaN   

   Vital_Respiratory Rate_mean_val  
0                              NaN  
1                            26.39  
2       

In [ ]:
bp_cols = ['Vital_Manual BP [Systolic]_max', 'Vital_Manual BP [Diastolic]_max', 'Vital_Arterial BP #2 [Systolic]_max',
           'Vital_Arterial BP #2 [Diastolic]_max', 'Vital_Arterial BP Mean #2_max',
            'Vital_ART BP mean_max', 'Vital_Arterial Blood Pressure systolic_max', 'Vital_Arterial Blood Pressure diastolic_max']
df[bp_cols].head()

for suf in ['_mean', '_min', '_max']:
    df['Vital_Mean Arterial Pressure Unified' + suf] = np.nan

    # arterial BP mean first
    df['Vital_Mean Arterial Pressure Unified' + suf] = df['Vital_Mean Arterial Pressure Unified' + suf].fillna(df['Vital_ART BP mean' + suf])
    # arterial BP mean #2
    df['Vital_Mean Arterial Pressure Unified' + suf] = df['Vital_Mean Arterial Pressure Unified' + suf].fillna(df['Vital_Arterial BP Mean #2' + suf])
    
    # arterial BP mean computed from arterial BP systolic and diastolic
    df['Vital_Mean Arterial Pressure Unified' + suf] = df['Vital_Mean Arterial Pressure Unified' + suf].fillna(
            (df['Vital_Arterial Blood Pressure systolic' + suf] + 2 * df['Vital_Arterial Blood Pressure diastolic' + suf]) / 3
        )
    # arterial mean computed from arterial BP systolic and diastolic #2
    df['Vital_Mean Arterial Pressure Unified' + suf] = df['Vital_Mean Arterial Pressure Unified' + suf].fillna(
            (df['Vital_Arterial BP #2 [Systolic]' + suf] + 2 * df['Vital_Arterial BP #2 [Diastolic]' + suf]) / 3
        )
    
    # arterial mean computed from manual BP systolic and diastolic 
    df['Vital_Mean Arterial Pressure Unified' + suf] = df['Vital_Mean Arterial Pressure Unified' + suf].fillna(
            (df['Vital_Manual BP [Systolic]' + suf] + 2 * df['Vital_Manual BP [Diastolic]' + suf]) / 3
        )

    df.drop(columns=['Vital_Manual BP [Systolic]' + suf, 'Vital_Manual BP [Diastolic]' + suf, 'Vital_Arterial BP #2 [Systolic]' + suf,
           'Vital_Arterial BP #2 [Diastolic]' + suf, 'Vital_Arterial BP Mean #2' + suf,
            'Vital_ART BP mean' + suf, 'Vital_Arterial Blood Pressure systolic' + suf, 'Vital_Arterial Blood Pressure diastolic' + suf], inplace=True)

,Vital_Manual BP [Systolic]_max,Vital_Manual BP [Diastolic]_max,Vital_Arterial BP #2 [Systolic]_max,Vital_Arterial BP #2 [Diastolic]_max,Vital_Arterial BP Mean #2_max,Vital_ART BP mean_max,Vital_Arterial Blood Pressure systolic_max,Vital_Arterial Blood Pressure diastolic_max
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,116.0,42.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
missing_threshold = 85

missing_pct = df.isna().mean() * 100
cols_to_drop = missing_pct[missing_pct >= missing_threshold].index
print(f"{len(cols_to_drop)} columns dropped due to high missing values (>{missing_threshold}%):\n")
print(list(cols_to_drop))

27 columns dropped due to high missing values (>85%):

['Vital_ART BP mean_max_val', 'Vital_Arterial BP #2 [Diastolic]_max_val', 'Vital_Arterial BP #2 [Systolic]_max_val', 'Vital_Arterial BP Mean #2_max_val', 'Vital_Manual BP Mean(calc)_max_val', 'Vital_Manual BP [Diastolic]_max_val', 'Vital_Manual BP [Systolic]_max_val', 'Vital_Temperature C_max_val', 'Vital_Temperature Celsius_max_val', 'Vital_ART BP mean_mean_val', 'Vital_Arterial BP #2 [Diastolic]_mean_val', 'Vital_Arterial BP #2 [Systolic]_mean_val', 'Vital_Arterial BP Mean #2_mean_val', 'Vital_Manual BP Mean(calc)_mean_val', 'Vital_Manual BP [Diastolic]_mean_val', 'Vital_Manual BP [Systolic]_mean_val', 'Vital_Temperature C_mean_val', 'Vital_Temperature Celsius_mean_val', 'Vital_ART BP mean_min_val', 'Vital_Arterial BP #2 [Diastolic]_min_val', 'Vital_Arterial BP #2 [Systolic]_min_val', 'Vital_Arterial BP Mean #2_min_val', 'Vital_Manual BP Mean(calc)_min_val', 'Vital_Manual BP [Diastolic]_min_val', 'Vital_Manual BP [Systolic]_min_v